In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pickle
import pandas as pd
from datetime import datetime

# Project Lists

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.ma   ximize_window()
# Extract link and project name
i = 1
while True:
    try:
        website = "https://cryptorank.io/funding-rounds?page={0}".format(i)
        
        driver.get(website)
        time.sleep(2)
        projects = driver.find_elements(By.XPATH,'//tr[@class="sc-7ff8d1ea-0 kuuWTw init-scroll"]')
        
        for project in projects:    
            name = project.find_element(By.TAG_NAME, "p").text
            token_sale_link = project.find_element(By.TAG_NAME, "a").get_attribute("href")
            overview_link = token_sale_link.replace("/ico/","/price/")
            
            data.append([name, overview_link, token_sale_link])
        i += 1
    except:
        break

df = pd.DataFrame(data, columns=['First Funding date', 'Overview', 'Token Sale'])
df['Name'] = df['Overview'].apply(lambda x: x.split("/")[-1]).apply(lambda x: x.split("#")[0]).apply(lambda x: x.replace("-", " ").title())

def convert_date(date_str):
    if len(date_str.split()) == 2:
        date_str += " 2024"
    try:
        dt = datetime.strptime(date_str, '%d %b %Y')
        return dt.strftime('%Y-%m-%d')
    except ValueError:
        return None

df['First Funding date'] = df['First Funding date'].apply(convert_date)
df.drop_duplicates().to_csv("project_lists.csv", index = False)

# Project Information

## Overview

In [230]:
df = pd.read_csv("project_lists.csv")
df.head()

0               https://cryptorank.io/price/raiinmaker
1    https://cryptorank.io/price/satoshisync#fundin...
2               https://cryptorank.io/price/legitimate
3                  https://cryptorank.io/price/mangata
4    https://cryptorank.io/price/bonus-block#fundin...
Name: Overview, dtype: object

In [264]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Extraction of link and project name ends here

services = []
total_raised_s = []
descriptions = []
links_s = []
name = []

for website in df['Overview'][:50]:
    crypto_name = website.split("/")[-1].split("#")[0].replace("_"," ").title()
    name.append(crypto_name)
    
    driver.get(website)
    time.sleep(5)
    service, total_raised, description, links = '', '', '', []
    try:
        service_element = driver.find_element(By.XPATH,'//a[starts-with(@href, "/categories/")]/p')
        services.append(service_element.text)
    except:
        # print(crypto_name," help1")
        pass
    try:
        total_raised_element = driver.find_element(By.XPATH,'//p[@class="sc-50f3633f-0 gsUEex"]')
        total_raised_s.append(total_raised_element.text)
    except:
        # print(crypto_name, " help2")
        pass
    try:
        description_element = driver.find_element(By.XPATH,'//div[@class="sc-50f3633f-0 sc-f084fb54-0 eEpKHS gQIZzl"]/p')
        descriptions.append(description_element.text)
    except:
        # print(crypto_name, " help3")
        pass
    
    
    ###########################
    
    try:
        button = driver.find_elements(By.XPATH,'//button[@class="sc-4e8da034-0 cHRla-d"]')
        for i in range(len(button)):
            driver.get(website)
            time.sleep(5)
            button = driver.find_elements(By.XPATH,'//button[@class="sc-4e8da034-0 cHRla-d"]')
            link = button[i]
            time.sleep(2)
            link.click()
            time.sleep(5)
            link_elements = driver.find_elements(By.XPATH,'//div[@class="sc-f30b0320-0 cAUFgN"]')
            heading = []
            some_links_text = []
            some_links = []
            for link_element in link_elements:
                some_links.append([i.get_attribute("href") for i in link_element.find_elements(By.TAG_NAME,"a")])
            links.append([some_links])
        links_s.append(links)
    except:
        # print(crypto_name, " help4")
        pass
    
    ############################

    time.sleep(10)
    print(len(name),len(services),len(total_raised_s),len(descriptions),len(links_s))

driver.quit()

1 1 1 1 1
Satoshisync  help2
Satoshisync  help4
2 2 1 2 1
3 3 2 3 2
4 4 3 4 3
Bonus-Block  help2
Bonus-Block  help4
5 5 3 5 3
Iotex  help2
Iotex  help4
6 6 3 6 3
Autolayer  help4
7 7 4 7 3
8 8 5 8 4
9 9 6 9 5
10 10 7 10 6
11 11 8 11 7
12 12 9 12 8
13 13 10 13 9
Metados  help2
Metados  help4
14 14 10 14 9
15 15 11 15 10
Bsx  help2
16 16 11 16 11
17 17 12 17 12
18 18 13 18 13
Merkle-Trade  help4
19 19 14 19 13
20 20 15 20 14
21 21 16 21 15
Biconomy  help2
Biconomy  help4
22 22 16 22 15
23 23 17 23 16
24 24 18 24 17
Taproot-Labs  help2
25 25 18 25 18
26 26 19 26 19
27 27 20 27 20
28 28 21 28 21
Hive  help3
29 29 22 28 22
30 30 23 29 23
Bloodloop  help4
31 31 24 30 23
32 32 25 31 24
33 33 26 32 25
Illuvium  help2
Illuvium  help4
34 34 26 33 25
35 35 27 34 26
36 36 28 35 27
Neutronpay  help3
37 37 29 35 28
38 38 30 36 29
39 39 31 37 30
Kimafinance  help4
40 40 32 38 30
41 41 33 39 31
42 42 34 40 32
43 43 35 41 33
44 44 36 42 34
45 45 37 43 35
Alex-Lab  help2
Alex-Lab  help4
46 46 37 44 35
B

In [271]:
data = [name,services,total_raised_s,descriptions,links_s]
df_overview = pd.DataFrame(data).T
df_overview.columns = ["Crypto Name", "Service", "Total Raised", "Description", "Links"]
df_overview.head()
df_overview.to_csv("Overview_details.csv", index=False)

## Fundraising

In [272]:
df = pd.read_csv("project_lists.csv")
df.head()

,First Funding date,Overview,Token Sale,Name
0,2024-04-03,https://cryptorank.io/price/raiinmaker,https://cryptorank.io/ico/raiinmaker,Raiinmaker
1,2024-04-03,https://cryptorank.io/price/satoshisync#fundin...,https://cryptorank.io/ico/satoshisync#funding-...,Satoshisync
2,2024-04-03,https://cryptorank.io/price/legitimate,https://cryptorank.io/ico/legitimate,Legitimate
3,2024-04-03,https://cryptorank.io/price/mangata,https://cryptorank.io/ico/mangata,Mangata
4,2024-04-02,https://cryptorank.io/price/bonus-block#fundin...,https://cryptorank.io/ico/bonus-block#funding-...,Bonus Block


In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

data = []
for website in df['Token Sale'][:50]:
    crypto_name = website.split("/")[-1].split("#")[0].replace("_"," ").title()
    driver.get(website)
    item_elements = driver.find_elements(By.XPATH,'//div[@class="sc-e007581-0 sc-b608d829-0 kDrqot bslrEt"]')
    data.append([crypto_name,[div.text for div in item_elements]])
    time.sleep(5)

df_fundings = pd.DataFrame(data, columns = ["Name", "Funding Details"])
df_fundings.to_csv("Funding_details.csv", index=False)

driver.quit()

Twitter

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(twitter_url) 
followers = driver.find_element(By.XPATH,'//a[contains(@href, "/verified_followers")]').text
followers

driver.quit()